# 📌ACTIVIDAD 2: UN CASO MÁS REAL.

Crea el fichero u02_p03_a2_reglin_<tus_iniciales>.py. Vamos a trabajar con los datos del fichero
"seguros_de_coches.csv" que utiliza una compañía aseguradora. <br><br>
COMPRENDER LOS DATOS DEL DATASET<br><br>
Hay 3 tipos de características:<br><br>
(a) La especificación de un automóvil en términos de diferentes características.<br><br>
(b) Su calificación de riesgo de seguro asignada. Es un indicador del grado en que el
automóvil es más problemático de lo que indica su precio. Inicialmente, a los automóviles se
les asigna un símbolo de factor de riesgo asociado con su precio. Luego, si es más arriesgado (o
menos), este símbolo se ajusta moviéndolo hacia arriba (o hacia abajo) en la escala. Los
profesionales llaman a este proceso “symboling”. Un valor de 3 indica que el automóvil es
riesgoso y -3 que probablemente sea bastante seguro.<br><br>
(c) El pago medio relativo por pérdida por año de vehículo asegurado. Este valor está
normalizado para todos los automóviles dentro de una clasificación de tamaño particular (twodoor small, station wagons, sports/speciality, etc…) y representa la pérdida promedio por
automóvil por año.<br><br>
Hay que tener en cuenta que los valores ausentes están definidos con un “?”. Completa los siguientes
pasos en el fichero Python comenzando por:<br><br>
a) Cargar los datos en un Dataframe, (adapta la ruta al código de la siguiente figura). <br><br>
b) Mostrar los 5 primeros ejemplos por pantalla (completa y obtén mismos resultados).<br><br>
c) Muestra un resumen de las columnas del dataset (completa y obtén mismos resultados). <br><br>


In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

columnas = ['symboling', 'perdidas_normal', 'marca', 'combustible', 'inyección',
            'puertas', 'chasis', 'traccion', 'lugar_de_motor', 'potencia_base',
            'longitud', 'anchura', 'altura', 'peso', 'tipo_motor',
            'cilindros', 'tamaño_motor', 'sistema', 'calibre', 'ataque',
            'ratio_compresion', 'potencia_cv', 'max_rpm', 'consumo_carretera', 'consumo_ciudad',
            'precio']

autos = pd.read_csv("seguros_de_coches.csv", header=None, names=columnas, na_values='?', delimiter=",")

DATOS AUSENTES Y CATEGÓRICOS<br><br>
Los datos se han cargado remplazando los símbolos “?” como valores ausentes NaN. como estaba
definido en la información del dataset. Primero sumarizamos los datos ausentes por característica.
Luego nos podemos fijar en el listado que nos ha devuelto el método autos.info() y todas las que no
tengan un tipo numérico (intX y floatX) serán categóricas de una u otra forma

In [ ]:
# Valores ausentes y columnas categóricas
print("========== Valores ausentes:\n", autos.isna().sum())

cols_categoricas = ["marca", "sistema"]  # Completa el array!!!

autos[cols_categoricas] = autos[cols_categoricas].astype("category")

autos["puertas"] = pd.Categorical(autos["puertas"], categories=["two", "four"], ordered=True)

autos["cilindros"] = pd.Categorical(autos["cilindros"],
                                    categories=["two", "three", "four", "five", "six", "eight", "twelve"],
                                    ordered=True)

print("========== Cambiamos a tipo category:\n", autos.info())

Para corregirlas podemos crearnos un array con sus nombres.
Completa el array cols_categoricas de la figura con el resto de
características categóricas (ahora solamente aparecen la primera y
la última). Usamos este array para cambiarles el tipo de dato. Y a
dos características ordinales le indicamos que lo son. Por último
mostramos información del dataframe. Deberías obtener algo
similar a esto como salida.<br><br>
Descripción estadística<br><br>
Se debe hacer un análisis de cada una de las variables y describir
sus propiedades. Realizar el análisis univariado es muy
importante para entender el comportamiento de cada una de las
variables y poder detectar posibles problemas en los datos. Nunca
hay que saltarse este paso. Por ejemplo vas a contar cuantas veces
aparece cada valor de las columnas categóricas. Aquellas
columnas que tengan valores que solamente aparezcan una vez,
las añades al array cols_categoricas_escasas.

In [ ]:
cols_categoricas_escasas = []  # Completa el array

for col in cols_categoricas:
    print(autos[col].value_counts())
    print("")

Algunos valores categóricos solo aparecen en un único ejemplo, es el caso de la característica “marca”
donde “mercury” solamente aparece una vez y eso puede dar problemas a los encoders. Igual ocurre
con otras características. Una solución es ocuparse de estos casos en la fase de ingeniería de
características (es lo que haremos en este ejemplo). Pero se puede perder conocimiento.
Otra solución es añadir más datos (posibilidad que no tenemos) o encargarse de garantizar que
siempre estén estos ejemplos en el conjunto de entrenamiento. Este procedimiento necesita una
implementación más compleja. Podemos crearnos un dataset temporal y añadirlo a los datos de train
los ejemplos con valores escasos de manera artificial. Para ello creamos una condición para cada
característica que tenga estos valores escasos y podemos generar sus datos. Completa las dos
condiciones que faltan del siguiente código:

In [ ]:
cond_1 = "sistema in ['mfi', 'spfi']"
cond_2 = "cilindros in [2,3]"
cond_3 = "cilindros in [4,5]"
unir_a_train = autos.query(f"{cond_1} | {cond_2} | {cond_3}")
print("Datos qe no sabemos perder:", unir_a_train)

ESTUDIAR ESTADÍSTICAS DE PAREJAS DE VARIABLES<br><br>
En primer lugar vamos a estudiar si todas las características numéricas influyen de manera lineal en el
target. Podemos hacerlo de manera visual creando scatterplot() de cada una con el target

In [ ]:
#====== Análisis bivariable
# generar listadp de variables númericas excepto target
cols_numericas = (autos.drop(columns=["precio"].select_dtypes(include=[np.number])).columns.tolist())
n_cols_numericas = len(cols_numericas)
import seaborn as sns
import math
fig, ejes = plt.subplots(math.ceil(n_cols_numericas / 5), 5, figsize=(10, 5))
ejes = ejes.flatten()
for i, col in enumerate(cols_numericas):
    sns.regplot(data=autos, x=col, y="precio", ax=ejes[i])
    ejes[i].set_title(col)
plt.tight_layout()
plt.show()

Estudiar la correlación entre variables<br><br>
La correlación lineal entre las variables predictoras que sean numéricas. Visualmente se puede ver en
las graficas como las anteriores pero realizadas entre cada par de predictoras o podemos hacer el
estudio de manera numérica creando una matriz de correlaciones. Vamos a usar esta segunda
aproximación, pero antes:<br><br>
• La variable "tamaño_motor" tiene una relación lineal positiva con el target.<br><br>
• Pero "symboling" no tiene relación lineal y por tanto el valor de la correlación no es válido.<br><br>
Como estamos interesados en la cantidad de correlación y no en si esta es positiva o negativa, lo que
vamos a hacer es generar la matriz de correlaciones y transformarla en sus valores absolutos para
visualizar mejor los posibles casos a contemplar. Primero ponemos la característica target la primer
apara que la primera fila y la primera columna se vean claramente. Debería aparecer lo más oscura
posible. Si es inferior a 0.1 (color claro cercano al blanco) es candidata a eliminarla como predictora.<br><br>
Además, si encontramos correlaciones importantes con otras predictoras es que hay columnas con
colinealidad y eso tampoco es bueno, podemos eliminar la que tenga menor correlación con target.

In [ ]:
from matplotlib.colors import Colormap as cm
cols = autos.columns.tolist()
cols.insert(0, cols.pop(cols.index('precio')))
autos = autos.reindex(columns=cols)
auto_matriz_correlaciones = autos.corr(numeric_only=True)
fig, ejes = plt.subplots(figsize=(12, 10))
absoluta = auto_matriz_correlaciones.abs()
sns.heatmap(absoluta, annot=True, annot_kws={'size':6}, fmt=".2f", cmap="Greens")

Es el caso de las características longitud, anchura, peso y tamaño del motor. Todas ellas tienen una alta
correlación. Serían candidatas a examinarlas en profundidad para eliminar algunas. <br><br>
A las características que tienen valores categóricos no podemos calcularles el coeficiente de
correlación pero sí podemos hacerles otros test estadísticos como ANOVA (Análisis de la Varianza). Pero
vamos simplemente a dibujar sus boxplot

In [ ]:
# Gráficos boxplot de categoricas con el target
n_categoricas = len(cols_categoricas)
fig, ejes = plt.subplots(n_categoricas // 4 + 1, 4, figsize=(12, 7))
ejes = ejes.flatten()
for i, col in enumerate(cols_categoricas):
    sns.boxplot(data=autos, x="precio", y=col, ax=ejes[i])
    ejes[i].set_title(col)
plt.tight_layout()
plt.show()

Hay variables categóricas que permiten distinguir entre grupos de valores de la variable objetivo, por
ejemplo la variable "lugar_de_motor" los dos boxplots tienen diferencias significativas para cada valor,
por lo que es una variable que sí aporta información al modelo. <br><br>
La variable "puertas" no permite distinguir claramente grupos de valores del target, por lo que no es
una variable que aporte información al modelo. Visualmente puedes identificar que la distribución de
los boxplots son similares para ambos valores: two ó four.

Aunque se detectan características a eliminar, en este ejemplo, continuamos trabajando con ellas.


FASE DE INGENIERÍA DE CARACTERÍSTICAS<br><br>
Aplicamos una imputación simple sin hacer un análisis mas profundo de los datos. Las variables
numéricas se imputan con la media y las categóricas con la moda. Usamos pipelines de
transformación porque nos dan flexibilidad y nos van a ahorrar mucho trabajo a la larga:<br><br>
• OneHotEncoder para las variables categóricas nominales.<br><br>
• OrdinalEncoder para las variables categóricas ordinales<br><br>

In [ ]:
#========== Ingeniería de características
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

y = autos['precio']
X = autos.drop('precio', axis='columns')
# Distribuir columnas predictoras en 1 de estas categorías
cols_categoricas = ["marca", "combustible", "inyeccion", "chasis", "traccion", "lugar_de_motor", "tipo_motor", "sistema"]
cols_categoricas_ordinales = ["puertas", "cilindros"]
cols_todas = X.columns.tolist()
cols_numericas = [x for x in cols_numericas if x not in cols_categoricas and x not in cols_categoricas_ordinales]
print("Predictoras ordinales: ", cols_categoricas_ordinales)
print("Predictoras catégoricas: ", cols_categoricas)
print("Predictoras números: ", cols_numericas)

pipe_numericas = Pipeline(steps[('imputer', SimpleImputer(strategy="median")), ('scaler', StandardScaler())])
pipe_categoricas = Pipeline(steps=[('imputer', SimpleImputer(strategy="most_frequent")), ('onehot', OneHotEncoder(handle_unknown="ignore"))])
pipe_categoricas_ordinales = Pipeline(steps=[
    ('ordenc', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)),
    ('imputer', SimpleImputer(strategy="most_frquent"))
])
preprocesa_columnas = ColumnTransformer(transformers=[
    ('numericas', pipe_numericas, cols_numericas),
    ('categoricas', pipe_categoricas, cols_categoricas),
    ('categoricas ordinales', pipe_categoricas_ordinales, cols_categoricas_ordinales),
])

DIVIDIR EL DATASET EN TRAIN Y TEST<br><br>
En el exterior, dejamos el 80% de los datos para entrenar y el 20% para test. No tenemos restricciones
de balanceo, etc. Si acaso, vendría bien escalar características numéricas según los algoritmos que
usemos

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print("Dimensiones de train: ",x_train.shape)
print("Dimensiones de test: ",x_test.shape)

BUSCAR UN BUEN MODELO DE REGRESIÓN<br><br>
Si se tienen muchos modelos, realizar validación cruzada con todos es costoso, así que se deben ir
descartando los modelos con menos desempeño hasta llegar al modelo final. Una forma de hacerlo:<br><br>
• Inicialmente se dividen los datos en dos partes: Una para realizar la selección del modelo
(datos de selección del modelo) y otra para realizar la prueba de desempeño (datos de test).
Esta parte de los datos se debe usar solo en el final de todo el proceso.<br><br>
• Luego se hace una evaluación de todos los modelos con la división anterior y se seleccionan los
mejores (es preferible usar modelos con principios de funcionamiento diferentes entre ellos).<br><br>
• Con lo mejores modelos (la cantidad depende de los resultados) se realiza la validación
cruzada (detectar si hay overfitting) para obtener los que tengan mejor resultado. Se saca el
mejor o los mejores modelos (mejor desempeño y poca varianza) y se realiza optimización de
hiperparámetros. Este proceso es costoso, por eso se debe realizar con muy pocos modelos.<br><br>
• Luego se selecciona el mejor modelo (mejor desempeño y menor varianza) y se obtienen los
hiperparámetros que dieron el mejor resultado.<br><br>
• Finalmente, se entrena el modelo seleccionado con los hiperparámetros encontrados con los
datos de selección de modelos y se hace la prueba con los datos de test.<br><br>
Creamos una función que se encargue de entrenar a un modelo. Esta función dividirá los datos que le
pasemos de nuevo en train y test. Aunque normalmente le pasaremos un x_train e y_train externo:

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures # Generar datos de potencias de los originales
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings("ignore")

dic_resultados = {}

def entrenar_modelo(modelo, procesador, X: pd.DataFrame, y: pd.Series, pct_test:float=0.2):
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=pct_test, random_state=123)
    pipe = Pipeline(steps[("preprocessor", procesador), ("model", modelo)])

Hacemos otra función para mostrar los resultados del entrenamiento:

Ahora creamos y entrenamos varios modelos de regresión lineal con la función que hemos definido y
registramos los score de cada uno en el diccionario creado a tal fin. Para el caso del modelo de
regresión polinomial, vamos a generar características nuevas así que para automatizar el proceso,
creamos un pipeline que automatice este nuevo paso, de esta manera no tendremos que hacerlo
nosotros manualmente:

Ahora podemos dibujar. Pero antes una aclaración, el modelo etiquetado como “Dummy” es uno que
directamente devuelve siempre un valor fijo independientemente de la entrada, así que solamente acierta si se le
aparece la virgen en un patinete. Como eso es improbable, es muy malo, en regresión similar a uno aleatorio. De
hecho, uno que sea peor que él, tiene mérito. Y sin embargo, uno de los que tenemos tiene tanto overfitting que
es mucho peor, porque al menos el dummy da la mediana y no se aleja mucho en sus predicciones, tendrá un
score negativo (coeficiente de determinación malo) pero no muy alto. Así que cuando ejecutes el código, el
modelo que de un score horrible es candidato a descartarlo (aunque si antes hubiésemos realizado limpieza de
predictoras, selección de características formalmente hablando, el resultado podría cambiar).

Vuelve a ejecutar el código, y si vuelve a aparecer uno que destaque por lo malo que es, lo volvemos a nominar y
comentamos su código.


Al final el gráfico que te quedará parecido al de arriba, verás que son todos bastante competentes (salvo el
dummy, claro y alguno que tenga overfitting). He tachado los nombres para que tengas que averiguar tu mismo
cuales tienen problemas si mantenemos todas las predictoras. Nos hemos quedado con 4 que parecen mejores.

VALIDACIÓN CRUZADA<br><br>
Analizar la varianza de los modelos más prometedores para obtener los que tengan mejor desempeño.
Aunque tu debes usar los 4 que hayas considerado mejores, yo te voy a mostrar el código de todos:


Ahora vamos a definir estructuras que nos permitan realizar la CV con 10-Fold para cada modelo. Yo
voy a usar todos y eso hará que mis gráficos no sean muy buenos al tener dos modelos muy malos.

Comparación Estadística de Modelos<br><br>
Ahora tras la validación cruzada vamos a usar los resultados para comprobar si hay o no una
diferencia significativa entre los modelos considerados. Usaremos un test ANOVA de un factor

En mi caso el test no es adecuado porque una de las condiciones que asume es que las desviaciones de
todos los datos que participan en el el test son similares (homoscedasticidad) y en mi ejecución no se
cumple al mantener los dos modelos que son rematadamente malos con una varianza excesiva
respecto a los demás.